<a href="https://colab.research.google.com/github/soruzo/megasena-linear-regression/blob/main/megasena_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import itertools
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping


In [4]:
def to_binary_matrix(lst):
    binary = np.zeros((len(lst), 60))
    for i, num in enumerate(lst):
        binary[i][num - 1] = 1
    return binary

In [15]:
# Lendo o arquivo
df = pd.read_excel('data.xlsx', header=6)

# Ajuste os índices das colunas conforme necessário
df_resultados = df.iloc[:, 2:8]

# Convertendo os dados para inteiros
df_resultados_int = df_resultados.apply(pd.to_numeric, errors='coerce').dropna()

# Convertendo os dados para uma matriz numpy
data = df_resultados_int.values.astype(int)

# Convertendo os dados de entrada em uma matriz bin
binary_data = to_binary_matrix(list(itertools.chain.from_iterable(data)))

# Separando os dados de entrada (X) dos dados de saída (y)
X = binary_data[:-1]
y = binary_data[1:]

      bola 1  bola 2  bola 3  bola 4  bola 5  bola 6
0         46      30      41      27       1      57
1          4      46      21      51       8       1
2         29       5      43      47      30      25
3          3      39      21      22      14      37
4         15      40      17      12      52      30
...      ...     ...     ...     ...     ...     ...
2663       1      19      46       6      16       2
2664       6      59      42      27       1       5
2665      36      30      10      11      29      47
2666       9      39      37      49      43      41
2667      41       5       4      52      30      33

[2668 rows x 6 columns]


In [16]:
# Criando o modelo da rede neural
model = Sequential()
model.add(Dense(128, input_dim=60, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(60, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Adicionando Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Treinando o modelo com validação e early stopping
model.fit(X, y, epochs=100, batch_size=32, verbose=1, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
401/401 [==============================] - 4s 6ms/step - loss: 4.0964 - accuracy: 0.0194 - val_loss: 4.0956 - val_accuracy: 0.0153
Epoch 2/100
401/401 [==============================] - 2s 5ms/step - loss: 4.0868 - accuracy: 0.0212 - val_loss: 4.1000 - val_accuracy: 0.0178
Epoch 3/100
401/401 [==============================] - 1s 4ms/step - loss: 4.0766 - accuracy: 0.0257 - val_loss: 4.1071 - val_accuracy: 0.0137
Epoch 4/100
401/401 [==============================] - 2s 4ms/step - loss: 4.0643 - accuracy: 0.0303 - val_loss: 4.1114 - val_accuracy: 0.0144
Epoch 5/100
401/401 [==============================] - 2s 4ms/step - loss: 4.0550 - accuracy: 0.0285 - val_loss: 4.1254 - val_accuracy: 0.0172
Epoch 6/100
401/401 [==============================] - 1s 4ms/step - loss: 4.0429 - accuracy: 0.0305 - val_loss: 4.1373 - val_accuracy: 0.0137
Epoch 7/100
401/401 [==============================] - 3s 6ms/step - loss: 4.0353 - accuracy: 0.0306 - val_loss: 4.1404 - val_accuracy: 0.0150

In [ ]:
# Gerando as próximas possíveis 10 sequências de 6 números
test_data = [data[-1]]
for i in range(5):
    # Convertendo os dados de entrada em uma matriz binária
    binary_test_data = to_binary_matrix(list(itertools.chain.from_iterable(test_data)))
    # Prevendo a próxima sequência de 6 números
    prob = model.predict(binary_test_data[-1].reshape(1, 60))
    next_seq = list(np.argsort(prob[0])[::-1][:6] + 1)
    # Adicionando a próxima sequência à lista de test_data
    test_data.append(next_seq)

# Imprimindo as próximas sequências
for i, seq in enumerate(test_data[1:]):
    print(f"Próxima sequência {i + 1}: {seq}")

1/1 [==============================] - 0s 29ms/step
Próxima sequência 1: [10, 24, 18, 40, 9, 51]
Próxima sequência 2: [37, 27, 31, 58, 34, 9]
Próxima sequência 3: [56, 41, 48, 30, 7, 43]
Próxima sequência 4: [28, 34, 48, 2, 5, 7]
Próxima sequência 5: [30, 42, 10, 18, 12, 40]
